In [124]:
import tensorflow

In [125]:
from tensorflow.keras.preprocessing import image

In [126]:
import cv2

In [127]:
import os

In [128]:
categories = ['withoutmask','withmask']

In [148]:
data = []
for category in categories:
    path  = os.path.join('data', category)
    
    label = categories.index(category)
    
    for file in os.listdir(path):
        img_path = os.path.join(path, file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        
        data.append([img,label])

In [149]:
import random

In [150]:
random.shuffle(data)

In [151]:
x=  []
y = []
for features,label in data:
    x.append(features)
    y.append(label)

In [152]:
import numpy as np

In [153]:
x = np.array(x)
y = np.array(y)

In [154]:
x=x/255

In [155]:
from sklearn.model_selection import train_test_split

In [156]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=.2)

In [157]:
from keras.applications.vgg16 import VGG16

In [158]:
vgg = VGG16()

In [159]:
from keras import Sequential

In [160]:
model  = Sequential()

In [161]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [162]:
for layer in model.layers:
    layer.trainable=False

In [163]:
from keras.layers import Dense

In [164]:
model.add(Dense(1,activation='sigmoid'))

In [165]:
from keras.layers import Dense

In [166]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [167]:
model.fit(X_train,y_train,epochs=5, validation_data=(X_test,y_test))

Epoch 1/5
25/25 [==============================] - 261s 11s/step - loss: 0.5863 - accuracy: 0.6662 - val_loss: 0.3804 - val_accuracy: 0.8750
Epoch 2/5
25/25 [==============================] - 248s 10s/step - loss: 0.3251 - accuracy: 0.9025 - val_loss: 0.2751 - val_accuracy: 0.9100
Epoch 3/5
25/25 [==============================] - 282s 11s/step - loss: 0.2483 - accuracy: 0.9362 - val_loss: 0.2380 - val_accuracy: 0.9150
Epoch 4/5
25/25 [==============================] - 250s 10s/step - loss: 0.2111 - accuracy: 0.9400 - val_loss: 0.2186 - val_accuracy: 0.9150
Epoch 5/5
25/25 [==============================] - 255s 10s/step - loss: 0.1898 - accuracy: 0.9463 - val_loss: 0.1842 - val_accuracy: 0.9350


In [168]:
import pickle

In [169]:
pickle.dump(model, open("model.pkl", "wb"))

INFO:tensorflow:Assets written to: ram://8987f33c-9d01-4fe2-9a5b-7dbbad104aa8/assets


INFO:tensorflow:Assets written to: ram://8987f33c-9d01-4fe2-9a5b-7dbbad104aa8/assets


In [170]:
import cv2

In [171]:
cap = cv2.VideoCapture(0)

In [199]:
while True:
    ret,frame = cap.read()
    #call the detection method
    img = cv2.resize(frame,(224,224))
    
    if detect_face_mask(img) >0.5:
        draw_label(frame,"WithMask",(30,30), (0,255,0))
    else:
        draw_label(frame,"Without Mask",(30,30), (255,0,0))
    
    
    cv2.imshow("window",frame)
    
    if (cv2.waitKey(10))%256==27:
        break
cv2.destroyAllWindows()

In [196]:
def detect_face_mask(img):
    y_predict =model.predict(img.reshape(1,224,224,3),verbose=None)
#     if y_predict > 0.5:
#         return "without_mask"
#     else:
#         return "with_mask"
    return y_predict[0][0]

In [191]:
img1 = cv2.imread(r'G:\MachileLearningProjects\facemaskdetector\data\with_mask_remaining\with_mask_6.jpg')
img1 = cv2.resize(img1,(224,224))


In [180]:
img2 = cv2.imread(r'G:\MachileLearningProjects\facemaskdetector\data\without_mask_remaining\without_mask_25.jpg')
img2 = cv2.resize(img2,(224,224))


In [193]:
img3 = cv2.imread(r'G:\MachileLearningProjects\facemaskdetector\data\without_mask_remaining\mine.jpg')
img3 = cv2.resize(img3,(224,224))
type(img3)

numpy.ndarray

In [175]:
detect_face_mask(img1)

1/1 [==============================] - 0s 314ms/step


0.9999736

In [176]:
detect_face_mask(img2)

1/1 [==============================] - 0s 208ms/step


0.0003843648

In [194]:
detect_face_mask(img3)

1/1 [==============================] - 0s 212ms/step


0.1257955

In [195]:
def draw_label(img, text, pos, bgcolor):
    text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    end_x = pos[0] + text_size[0][0]+2
    end_y = pos[1] + text_size[0][1] -2
    cv2.rectangle(img, pos, (end_x,end_y),bgcolor,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)
    